In [249]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [250]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [251]:
# libraries to build algorithm
import sys
import numpy as np
from numpy import linalg as la
# libraries for reading data
import pandas as pd
# libraries for plotting
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# libraries for in-built algorithm
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

In [252]:
# custom classes and functions
sys.path.append('..')

## Load data

In [253]:
df = pd.read_csv('ml-100k.csv')
df = df.iloc[:, :-1]
df.head()

user_id  item_id  rating
0      196      242       3
1      186      302       3
2       22      377       1
3      244       51       2
4      166      346       1

## Algorithm

In [254]:
class MF_MSE():
    '''inputs:
    X: (n, m)
    y: (n, 1)
    w: (m, 1) - weights'''
    def compute_error_matrix(self, y_hat, y, squared=True):
        '''compute sum of squared error excluding indices where y has missing data'''
        error_matrix = (y_hat - y)
        if squared:
            error_matrix = error_matrix**2
        error_matrix[np.isnan(y)] = 0
        return error_matrix
    
    def compute_cost(self, X, w, y, reg_lambda):
        y_hat = np.matmul(X, w)
        error_term = self.compute_error_matrix(y_hat, y, squared=True)
        X_reg_term = reg_lambda * X**2
        w_reg_term = reg_lambda * w**2      
        self.cost = ( np.sum(error_term) + np.sum(X_reg_term) + np.sum(w_reg_term) ) / 2
    
    def derivative(self, X, w, y, reg_lambda):
        y_hat = np.matmul(X, w)
        error_term = self.compute_error_matrix(y_hat, y, squared=False)
        X_reg_term = reg_lambda * X
        w_reg_term = reg_lambda * w
        dC_dX = np.matmul(error_term, w.T) + X_reg_term
        dC_dw = np.matmul(X.T, error_term) + w_reg_term
        return dC_dX, dC_dw
    
class MF_GradientDescent():
    def __init__(self, X, y, w, loss, reg_lambda = 0, seed = None):
        '''input:
        X: (n, m)
        y: (n, 1)
        loss: instace of class with at least two methods: "compute_loss" and "derivative"
        '''
        np.random.seed(seed)
        self.X = X
        self.w = w
        self.y = y
        # random weight initialization
        self.loss = loss
        self.reg_lambda = reg_lambda
        self.loss.compute_cost(self.X, self.w, self.y, self.reg_lambda)
    
    def jacobian(self):
        self.X_gradient, self.w_gradient = self.loss.derivative(self.X, self.w, self.y, self.reg_lambda)
    
    def weight_update(self, learning_rate):
        self.jacobian()
        self.X -= learning_rate * self.X_gradient
        self.w -= learning_rate * self.w_gradient
        
    def gradient_descent(self, learning_rate = 0.1, max_iter = 100, threshold = 1e-3, debug = False):
        if debug:
            print('initial weights {}'.format(self.w))
            print('initial X {}'.format(self.X))
        i = 0
        while(i < max_iter and self.loss.cost > threshold):
            if debug:
                print('\n****iter {}'.format(i+1))
            self.weight_update(learning_rate)
            if debug:
                print('w gradient {}'.format(self.w_gradient))
                print('weights {}'.format(self.w))
                print('X gradient {}'.format(self.X_gradient))
                print('X {}'.format(self.X))
            self.loss.compute_cost(self.X, self.w, self.y, self.reg_lambda)
            if debug:
                print('cost {}'.format(self.loss.cost))
            i += 1    

class LowRankFactorization():
    
    def __init__(self, learning_rate = 0.001, max_iter = 100, threshold = 0.01, reg_lambda = 0, debug = False, seed = None):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.threshold = threshold
        self.reg_lambda = reg_lambda
        self.debug = debug
        self.seed = seed
    
    def init_(self):
        self.item = np.random.randn(self.rating.shape[0], self.n_dim)/ 2 # shape: (n_items, n_dim)
        self.user = (np.random.randn(self.rating.shape[1], self.n_dim)/ 2).T # shape: (n_dim, n_users)
    
    def fit(self, rating, n_dim):
        '''rating is a rating matrix of shape (i,u) where i is number of items, u is number of users
        rating should be mean centered
        n_dim is the desired dimensionality of the latent feature space'''
        self.rating = rating
        self.n_dim = n_dim
        self.init_()
        # gradient descent with Sum of Squared Residuals as loss
        ss = MF_GradientDescent(X = self.item, y = self.rating, w = self.user, loss = MF_MSE(), reg_lambda = self.reg_lambda, seed = self.seed)
        ss.gradient_descent(learning_rate = self.learning_rate, 
                            max_iter = self.max_iter, 
                            threshold = self.threshold, 
                            debug = self.debug)
        self.item = ss.X
        self.user = ss.w
   
    def predict(self, add_mean = 0):
        '''returns ratings matrix with filled missing values'''
        return np.matmul(self.item, self.user) + add_mean            

## Compare with in-built algorithm

In [255]:
# Use the famous SVD algorithm.
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df, reader)
trainset, testset = train_test_split(data, test_size=0.25)
svd = SVD(n_factors = 100)
y_pred2 = svd.fit(trainset).test(testset)
y_pred2 = pd.DataFrame(y_pred2, columns=['uid', 'iid', 'rui', 'est', 'details']).iloc[:, :-1]

In [256]:
y_pred2.head()

uid  iid  rui       est
0  120  282  4.0  3.652481
1  882  291  4.0  3.732709
2  535  507  5.0  4.039265
3  697  244  5.0  3.247540
4  751  385  4.0  3.417123

In [257]:
len(y_pred2)

25000

In [258]:
mse2 = (y_pred2['rui']-y_pred2['est']).apply(lambda x: x**2).dropna().sum() / (y_pred2.shape[0] - y_pred2['rui'].isna().sum())
print('mse: {}'.format(mse2))

mse: 0.9004063693093578


## Apply algorithm

In [259]:
# prepare data. Use same test set as built-in algo
test_user_item = list(y_pred2['uid'].astype(str) + '-' + y_pred2['iid'].astype(str))
df['user_item'] = df['user_id'].astype(str) + '-' + df['item_id'].astype(str)
test = df.loc[df.user_item.isin(test_user_item)]
train = df.copy()
train.iloc[test.index, -1] = np.nan # mask test data with nan
print('n test ratings: {}'.format(len(test)))

n test ratings: 25000


In [260]:
X = df.sort_values('user_id').pivot(index = 'item_id', columns = 'user_id', values = 'rating').sort_values('item_id') # original data
X_train = train.sort_values('user_id').pivot(index = 'item_id', columns = 'user_id', values = 'rating').sort_values('item_id') # masked data
X_train.head()

user_id  1    2    3    4    5    6    7    8    9    10   ...  934  935  936  \
item_id                                                    ...                  
1        5.0  4.0  NaN  NaN  4.0  4.0  NaN  NaN  NaN  4.0  ...  2.0  3.0  4.0   
2        3.0  NaN  NaN  NaN  3.0  NaN  NaN  NaN  NaN  NaN  ...  4.0  NaN  NaN   
3        4.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  4.0   
4        3.0  NaN  NaN  NaN  NaN  NaN  5.0  NaN  NaN  4.0  ...  5.0  NaN  NaN   
5        3.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   

user_id  937  938  939  940  941  942  943  
item_id                                     
1        NaN  4.0  NaN  NaN  5.0  NaN  NaN  
2        NaN  NaN  NaN  NaN  NaN  NaN  5.0  
3        NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4        NaN  NaN  NaN  2.0  NaN  NaN  NaN  
5        NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 943 columns]

In [261]:
X_array = np.array(X)
X_train_array = np.array(X_train)
X_array.shape

(1682, 943)

In [262]:
# mean center X
X_train_mean = np.nanmean(X_train_array, axis=1).reshape(-1, 1)
X_train_array = X_train_array - X_train_mean

In [264]:
mf = LowRankFactorization(learning_rate = 0.001, max_iter = 1000, threshold = 0.001, reg_lambda = 0.5, debug = False, seed = 0)
mf.fit(rating = X_train_array, n_dim = 100)
y_pred = mf.predict(add_mean = X_train_mean)

In [265]:
y_pred_df = pd.DataFrame(y_pred, columns = X_train.columns, index = X_train.index).unstack().reset_index()
y_pred_df.columns = ['user_id', 'item_id', 'pred_rating']
y_pred_df.head()

user_id  item_id  pred_rating
0        1        1     5.462008
1        1        2     2.949447
2        1        3     3.925234
3        1        4     3.323778
4        1        5     2.856985

In [266]:
y_pred_df['user_item'] = y_pred_df['user_id'].astype(str) + '-' + y_pred_df['item_id'].astype(str)
test_y_pred = y_pred_df.loc[y_pred_df['user_item'].isin(test_user_item)]

In [267]:
test_y_pred = test_y_pred.merge(df[['user_id','item_id','rating']], on = ['user_id','item_id'])

In [268]:
mse = (test_y_pred['rating'] - test_y_pred['pred_rating']).apply(lambda x: x**2).dropna().sum() / (test_y_pred.shape[0] - test_y_pred['rating'].isna().sum())
print('mse: {}'.format(mse))

mse: 0.018051884567929867


In [269]:
test_y_pred.head()

user_id  item_id  pred_rating user_item  rating
0        1        1     5.462008       1-1       5
1        1        2     2.949447       1-2       3
2        1        4     3.323778       1-4       3
3        1        6     4.905204       1-6       5
4        1       17     3.195499      1-17       3